In [17]:
import numpy as np
import pulp
from typing import List, Tuple, Dict, Optional, Any
import warnings
import json


class GSDIJ_AHP:
    def __init__(self, debug_mode: bool = False):
        self.n = 0  # Количество критериев
        self.m = 0  # Количество экспертов
        self.individual_matrices: List[np.ndarray] = []  # Индивидуальные матрицы
        self.alpha_weights: List[float] = []  # Веса экспертов
        self.criteria_names: List[str] = []  # Названия критериев
        self.dm_ids: List[str] = []  # Идентификаторы экспертов
        self.lambda_opt: float = 0.0  # Оптимальное значение лямбды
        self.group_interval_matrix: Optional[Tuple[np.ndarray, np.ndarray]] = None
        self.group_weights: Optional[np.ndarray] = None
        self.debug_mode = debug_mode

    def load_from_json(self, filepath: str) -> None:
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Загрузка данных
            if 'criteria' in data:
                self.criteria_names = [criterion['name'] for criterion in data['criteria']]
                self.n = len(self.criteria_names)
            elif 'criteria' in data and isinstance(data['criteria'][0], str):
                self.criteria_names = data['criteria']
                self.n = len(self.criteria_names)
            else:
                raise ValueError("Не найдены названия критериев в JSON файле")
 
            if 'dms' not in data:
                raise ValueError("Не найдены данные экспертов в JSON файле")
            
            self.individual_matrices = []
            self.dm_ids = []
            
            for dm_data in data['dms']:
                dm_id = dm_data.get('id', f'DM{len(self.dm_ids)+1}')
                self.dm_ids.append(dm_id)
                
                if 'pairwise_comparisons' in dm_data:
                    matrix = np.array(dm_data['pairwise_comparisons'], dtype=float)
                    if matrix.shape != (self.n, self.n):
                        raise ValueError(f"Матрица эксперта {dm_id} имеет неверный размер: {matrix.shape}, ожидается ({self.n}, {self.n})")
                    self.individual_matrices.append(matrix)
                elif 'matrix' in dm_data:
                    matrix = np.array(dm_data['matrix'], dtype=float)
                    if matrix.shape != (self.n, self.n):
                        raise ValueError(f"Матрица эксперта {dm_id} имеет неверный размер")
                    self.individual_matrices.append(matrix)
                else:
                    raise ValueError(f"Не найдена матрица сравнений для эксперта {dm_id}")
            
            self.m = len(self.individual_matrices)
            
            # Загрузка весов экспертов
            if 'parameters' in data:
                params = data['parameters']
                if 'alpha_weights' in params:
                    weights = params['alpha_weights']
                    if len(weights) != self.m:
                        raise ValueError(f"Количество весов экспертов ({len(weights)}) не совпадает с количеством экспертов ({self.m})")
                    self.alpha_weights = [float(w) for w in weights]
                else:
                    self.alpha_weights = [1.0 / self.m] * self.m
            else:
                self.alpha_weights = [1.0 / self.m] * self.m
            
            # Нормализация весов (если нужно)
            total_weight = sum(self.alpha_weights)
            if total_weight != 1.0:
                self.alpha_weights = [w / total_weight for w in self.alpha_weights]
            
            if self.debug_mode:
                print(f"Успешно загружено из {filepath}")
                print(f"   Критериев: {self.n}")
                print(f"   Экспертов: {self.m}")
            
        except FileNotFoundError:
            raise FileNotFoundError(f"Файл {filepath} не найден")
        except json.JSONDecodeError as e:
            raise ValueError(f"Ошибка чтения JSON файла: {e}")
        except Exception as e:
            raise ValueError(f"Ошибка загрузки данных: {e}")

    def compute_wgmm_matrix(self) -> np.ndarray:
        #Вычисление взвешенной геометрической средней матрицы 
        wgmm_matrix = np.ones((self.n, self.n))

        for i in range(self.n):
            for j in range(self.n):
                if i == j:
                    continue
                product = 1.0
                for k in range(self.m):
                    product *= self.individual_matrices[k][i, j] ** self.alpha_weights[k]
                wgmm_matrix[i, j] = product

        return wgmm_matrix

    def compute_geometric_std_matrix(self, wgmm_matrix: np.ndarray) -> np.ndarray:
        #Вычисление матрицы геометрических стандартных отклонений

        std_matrix = np.ones((self.n, self.n))
        sum_alpha_sq = sum(a ** 2 for a in self.alpha_weights)
        denominator = max(1.0 - sum_alpha_sq, 1e-10)

        for i in range(self.n):
            for j in range(self.n):
                if i == j:
                    continue
                sum_sq = 0.0
                for k in range(self.m):
                    ratio = self.individual_matrices[k][i, j] / wgmm_matrix[i, j]
                    log_ratio = np.log(ratio)
                    sum_sq += self.alpha_weights[k] * (log_ratio ** 2)

                std_matrix[i, j] = np.exp(np.sqrt(sum_sq / denominator))

        return std_matrix

    def compute_group_interval_matrix(self, lambda_val: float) -> Tuple[np.ndarray, np.ndarray]:
        # Вычисление интервальной групповой матрицы
        wgmm_matrix = self.compute_wgmm_matrix()
        std_matrix = self.compute_geometric_std_matrix(wgmm_matrix)

        lower_matrix = np.ones((self.n, self.n))
        upper_matrix = np.ones((self.n, self.n))

        for i in range(self.n):
            for j in range(i + 1, self.n):
                wgmm_val = wgmm_matrix[i, j]
                std_val = std_matrix[i, j]

                lower_matrix[i, j] = wgmm_val / (std_val ** lambda_val)
                upper_matrix[i, j] = wgmm_val * (std_val ** lambda_val)
                
                lower_matrix[j, i] = 1.0 / upper_matrix[i, j]
                upper_matrix[j, i] = 1.0 / lower_matrix[i, j]

        return lower_matrix, upper_matrix

    def compute_indeterminacy_index(self, lower_matrix: np.ndarray, upper_matrix: np.ndarray) -> float:
        
        # Вычисление индекса неопределённости интервальной матрицы
        
        if self.n < 2:
            return 1.0
            
        product = 1.0
        count = 0

        for i in range(self.n):
            for j in range(i + 1, self.n):
                if lower_matrix[i, j] > 0:
                    ratio = upper_matrix[i, j] / lower_matrix[i, j]
                    product *= ratio
                    count += 1

        if count > 0:
            return product ** (2.0 / (self.n * (self.n - 1)))
        return 1.0

    def compute_group_satisfaction_index(self, lower_matrix: np.ndarray, upper_matrix: np.ndarray) -> float:
        
        #Вычисление индекса групповой удовлетворённости (GSI)
        if self.n < 2:
            return 1.0
            
        total_satisfaction = 0.0
        total_comparisons = self.n * (self.n - 1) // 2

        for k in range(self.m):
            count_satisfied = 0
            for i in range(self.n):
                for j in range(i + 1, self.n):
                    individual_value = self.individual_matrices[k][i, j]
                    if lower_matrix[i, j] <= individual_value <= upper_matrix[i, j]:
                        count_satisfied += 1

            satisfaction_k = count_satisfied / total_comparisons
            total_satisfaction += self.alpha_weights[k] * satisfaction_k

        return total_satisfaction

    def solve_model_1(self, t: float = 3.0, s: float = 0.5, n_points: int = 1001) -> float:
#Оптимизация параметра дямбда
        # Поиск минимального лямбды, удовлетворяющего ограничениям
        lambdas = np.linspace(0.0, 1.0, n_points)
        feasible_lambdas = []
        
        for lambda_val in lambdas:
            lower, upper = self.compute_group_interval_matrix(lambda_val)
            U = self.compute_indeterminacy_index(lower, upper)
            GSI = self.compute_group_satisfaction_index(lower, upper)
            
            if U <= t and GSI >= s:
                feasible_lambdas.append(lambda_val)
        
        # Если найдено решение, возвращаем минимальное лямбда
        if feasible_lambdas:
            return min(feasible_lambdas)
        
        # Если решение не найдено, ищем лямбду, при котором U = t
        if self.debug_mode:
            print("Model 1 не имеет решения. Ищем λ, при котором U = t")
        
        # Бинарный поиск лямбды, где U ≈ t
        lo, hi = 0.0, 1.0
        for _ in range(50):
            mid = (lo + hi) / 2
            lower, upper = self.compute_group_interval_matrix(mid)
            U = self.compute_indeterminacy_index(lower, upper)
            
            if U < t:
                lo = mid
            else:
                hi = mid
        
        lambda_fallback = (lo + hi) / 2
        
        if self.debug_mode:
            lower, upper = self.compute_group_interval_matrix(lambda_fallback)
            U_final = self.compute_indeterminacy_index(lower, upper)
            GSI_final = self.compute_group_satisfaction_index(lower, upper)
            print(f"λ = {lambda_fallback:.4f}, U = {U_final:.4f}, GSI = {GSI_final:.4f}")
        
        return lambda_fallback

    def solve_model_2_fpp(self, lower_matrix: np.ndarray, upper_matrix: np.ndarray) -> Tuple[np.ndarray, float]:
       # Решение Model 2: метод нечёткого программирования предпочтений (FPP)
        prob = pulp.LpProblem("FPP_Weights", pulp.LpMaximize)
        
        c = pulp.LpVariable("c", lowBound=0.0, upBound=1.0)
        w_vars = [pulp.LpVariable(f"w_{i}", lowBound=1e-8) for i in range(self.n)]
        
        prob += c
        prob += pulp.lpSum(w_vars) == 1.0
        
        for i in range(self.n):
            for j in range(i + 1, self.n):
                prob += w_vars[i] - w_vars[j] * upper_matrix[i, j] <= 1.0 * (1 - c)
                prob += -w_vars[i] + w_vars[j] * lower_matrix[i, j] <= 1.0 * (1 - c)

        solver = pulp.PULP_CBC_CMD(msg=False, timeLimit=30, gapRel=1e-10)
        prob.solve(solver)
        
        if pulp.LpStatus[prob.status] == "Optimal":
            c_value = c.varValue
            weights = np.array([w.varValue for w in w_vars])
            weights = weights / np.sum(weights)
            
            return weights, c_value
        else:
            raise RuntimeError(f"FPP не удался. Статус: {pulp.LpStatus[prob.status]}")

    def compute_weight_bounds(self, lower_matrix: np.ndarray, upper_matrix: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        #вычисление мин и макс возможных весов
        w_min = np.zeros(self.n)
        w_max = np.zeros(self.n)
        
        for k in range(self.n):
            # Минимальный вес
            prob_min = pulp.LpProblem(f"Min_w_{k}", pulp.LpMinimize)
            w_vars_min = [pulp.LpVariable(f"w_{i}_min", lowBound=1e-8) for i in range(self.n)]
            prob_min += w_vars_min[k]
            prob_min += pulp.lpSum(w_vars_min) == 1.0
            
            for i in range(self.n):
                for j in range(i + 1, self.n):
                    prob_min += w_vars_min[i] <= upper_matrix[i, j] * w_vars_min[j]
                    prob_min += w_vars_min[i] >= lower_matrix[i, j] * w_vars_min[j]
            
            prob_min.solve(pulp.PULP_CBC_CMD(msg=False))
            w_min[k] = pulp.value(w_vars_min[k])
            
            # Максимальный вес
            prob_max = pulp.LpProblem(f"Max_w_{k}", pulp.LpMaximize)
            w_vars_max = [pulp.LpVariable(f"w_{i}_max", lowBound=1e-8) for i in range(self.n)]
            prob_max += w_vars_max[k]
            prob_max += pulp.lpSum(w_vars_max) == 1.0
            
            for i in range(self.n):
                for j in range(i + 1, self.n):
                    prob_max += w_vars_max[i] <= upper_matrix[i, j] * w_vars_max[j]
                    prob_max += w_vars_max[i] >= lower_matrix[i, j] * w_vars_max[j]
            
            prob_max.solve(pulp.PULP_CBC_CMD(msg=False))
            w_max[k] = pulp.value(w_vars_max[k])
        
        return w_min, w_max

    def run_complete_analysis(self, t: float = 3.0, s: float = 0.5) -> Dict:
        wgmm_matrix = self.compute_wgmm_matrix()
        std_matrix = self.compute_geometric_std_matrix(wgmm_matrix)
        
        print("Матрица WGMM:")
        for i in range(self.n):
            row = "  [ "
            for j in range(self.n):
                row += f"{wgmm_matrix[i, j]:8.4f} "
            row += "]"
            print(row)
        
        print("\nМатрица геометрического стандартного отклонения:")
        for i in range(self.n):
            row = "  [ "
            for j in range(self.n):
                row += f"{std_matrix[i, j]:8.4f} "
            row += "]"
            print(row)

        print(f"\n2. Оптимизация параметра лямбда")
        print("-" * 40)
        
        self.lambda_opt = self.solve_model_1(t, s)
        lower_matrix, upper_matrix = self.compute_group_interval_matrix(self.lambda_opt)
        
        U = self.compute_indeterminacy_index(lower_matrix, upper_matrix)
        GSI = self.compute_group_satisfaction_index(lower_matrix, upper_matrix)
        
        print(f"Оптимальное лямбда: {self.lambda_opt:.4f}")
        print(f"Индекс неопределённости U: {U:.4f} {'✓' if U <= t else '✗'}")
        print(f"Индекс групповой удовлетворённости GSI: {GSI:.4f} {'✓' if GSI >= s else '✗'}")

        print(f"\n3. ИНТЕРВАЛЬНАЯ ГРУППОВАЯ МАТРИЦА")
        print("-" * 40)
        
        for i in range(self.n):
            row = "  [ "
            for j in range(self.n):
                if i == j:
                    row += "1.0000        "
                else:
                    row += f"[{lower_matrix[i, j]:6.4f},{upper_matrix[i, j]:6.4f}] "
            row += "]"
            print(row)

        print(f"\n4. ВЫЧИСЛЕНИЕ ВЕСОВ КРИТЕРИЕВ")
        
        try:
            weights, c_value = self.solve_model_2_fpp(lower_matrix, upper_matrix)
            
            if abs(c_value - 1.0) < 1e-6:
                w_min, w_max = self.compute_weight_bounds(lower_matrix, upper_matrix)
                weights = (w_min + w_max) / 2
                weights = weights / np.sum(weights)
            else:
                print(f"c* = {c_value:.6f}  - частичное выполнение ограничений")
            
            self.group_weights = weights
            self.group_interval_matrix = (lower_matrix, upper_matrix)
            
            print("\nФинальные групповые веса критериев:")
            for i, (name, w) in enumerate(zip(self.criteria_names, weights)):
                print(f"  {i+1:2d}. {name:20} : {w:.6f} ({w*100:.1f}%)")
            
            print(f"\n РАНЖИРОВАНИЕ КРИТЕРИЕВ ПО ВАЖНОСТИ")
            
            sorted_indices = np.argsort(weights)[::-1]
            for rank, idx in enumerate(sorted_indices):
                print(f"  Ранг {rank+1:2d}: {self.criteria_names[idx]:20} (вес = {weights[idx]:.6f})")
            
            return {
                'lambda_opt': self.lambda_opt,
                'U': U,
                'GSI': GSI,
                'weights': weights.tolist(),
                'weights_dict': {name: float(w) for name, w in zip(self.criteria_names, weights)},
                'ranking': [int(idx) for idx in sorted_indices.tolist()],
                'ranking_names': [self.criteria_names[idx] for idx in sorted_indices],
                'interval_matrix': {
                    'lower': lower_matrix.tolist(),
                    'upper': upper_matrix.tolist()
                },
                'c_value': float(c_value),
                'criteria_names': self.criteria_names,
                'dm_ids': self.dm_ids,
                'alpha_weights': self.alpha_weights
            }
            
        except Exception as e:
            print(f"Ошибка в FPP: {e}")


    def save_interval_and_ranking(self, filepath: str, interval_matrix: Optional[tuple] = None) -> Dict[str, Any]:
        if interval_matrix is None:
            interval_matrix = self.group_interval_matrix

        if interval_matrix is None:
            raise RuntimeError(" ")

        if self.group_weights is None:
            raise RuntimeError(" ")

        lower, upper = interval_matrix
        w = np.asarray(self.group_weights, dtype=float)
        
        order = np.argsort(w)[::-1]
        ranking = []
        for rank, idx in enumerate(order, start=1):
            ranking.append({
                "rank": int(rank),
                "criterion_index": int(idx),
                "criterion_name": self.criteria_names[idx] if idx < len(self.criteria_names) else str(idx),
                "weight": float(w[idx]),
            })

        payload = {
            "criteria_names": self.criteria_names,
            "weights": w.tolist(),                 # numpy -> list для JSON [web:196]
            "ranking": ranking,
            "interval_matrix": {
                "lower": np.asarray(lower).tolist(),  # numpy -> list для JSON [web:196]
                "upper": np.asarray(upper).tolist(),
            },
        }

        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(payload, f, ensure_ascii=False, indent=2)

        return payload

if __name__ == "__main__":
    
    # Создаем объект GSD-IJ
    gsd_ij = GSDIJ_AHP(debug_mode=True)
    
    # Загружаем данные 
    json_file_path = "pcm_output.json"  
    
    try:
        gsd_ij.load_from_json(json_file_path)
        
        # Запускаем анализ
        results = gsd_ij.run_complete_analysis(t=3, s=0.3)
        gsd_ij.save_interval_and_ranking("results_wgmm_example2.json")
        
        # Выводим результаты
        print(f"Оптимальный λ: {results['lambda_opt']:.4f}")
        print(f"Индекс неопределённости: {results['U']:.4f}")
        print(f"Индекс удовлетворённости: {results['GSI']:.4f}")
        print(f"Степень выполнения ограничений (c*): {results['c_value']:.4f}")
        
        print("\nРанжирование критериев:")
        for rank, name in enumerate(results['ranking_names'], 1):
            print(f"{rank}. {name}")
            
    except Exception as e:
        print(f"Ошибка: {e}")

Успешно загружено из pcm_output.json
   Критериев: 4
   Экспертов: 6
Матрица WGMM:
  [   1.0000   1.0587   1.7373   1.7201 ]
  [   0.9445   1.0000   1.6410   1.6247 ]
  [   0.5756   0.6094   1.0000   0.9901 ]
  [   0.5814   0.6155   1.0100   1.0000 ]

Матрица геометрического стандартного отклонения:
  [   1.0000   1.9784   2.1845   1.7062 ]
  [   1.9784   1.0000   1.9473   1.3832 ]
  [   2.1845   1.9472   1.0000   1.8767 ]
  [   1.7062   1.3832   1.8767   1.0000 ]

2. Оптимизация параметра лямбда
----------------------------------------
Оптимальное лямбда: 0.2970
Индекс неопределённости U: 1.4308 ✓
Индекс групповой удовлетворённости GSI: 0.3056 ✓

3. ИНТЕРВАЛЬНАЯ ГРУППОВАЯ МАТРИЦА
----------------------------------------
  [ 1.0000        [0.8645,1.2965] [1.3775,2.1911] [1.4677,2.0159] ]
  [ [0.7713,1.1567] 1.0000        [1.3463,2.0001] [1.4755,1.7890] ]
  [ [0.4564,0.7260] [0.5000,0.7428] 1.0000        [0.8213,1.1936] ]
  [ [0.4961,0.6813] [0.5590,0.6777] [0.8378,1.2177] 1.0000       

Совпадающих (t,s) не найдено на заданной сетке.
